Machine Learning para Jornalistas - Aula 7 - Segmentação

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

Importando os Dados - Votações na Câmara dos Deputados em 2021

In [ ]:
base_votos = pd.read_csv("votacoesVotos-2021.csv",sep=';')

Explorando a base de dados

In [ ]:
base_votos.columns

Index(['idVotacao', 'uriVotacao', 'dataHoraVoto', 'voto', 'deputado_id',
       'deputado_uri', 'deputado_nome', 'deputado_siglaPartido',
       'deputado_uriPartido', 'deputado_siglaUf', 'deputado_idLegislatura',
       'deputado_urlFoto'],
      dtype='object')

In [ ]:
base_votos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287671 entries, 0 to 287670
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   idVotacao               287671 non-null  object
 1   uriVotacao              287671 non-null  object
 2   dataHoraVoto            287671 non-null  object
 3   voto                    287671 non-null  object
 4   deputado_id             287671 non-null  int64 
 5   deputado_uri            287671 non-null  object
 6   deputado_nome           287671 non-null  object
 7   deputado_siglaPartido   287671 non-null  object
 8   deputado_uriPartido     287671 non-null  object
 9   deputado_siglaUf        287671 non-null  object
 10  deputado_idLegislatura  287671 non-null  int64 
 11  deputado_urlFoto        287671 non-null  object
dtypes: int64(2), object(10)
memory usage: 26.3+ MB


In [ ]:
base_votos.head()

,idVotacao,uriVotacao,dataHoraVoto,voto,deputado_id,deputado_uri,deputado_nome,deputado_siglaPartido,deputado_uriPartido,deputado_siglaUf,deputado_idLegislatura,deputado_urlFoto
0,14666-138,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-08-04T11:39:19,Sim,204545,https://dadosabertos.camara.leg.br/api/v2/depu...,Alê Silva,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,MG,56,http://www.camara.gov.br/internet/deputado/ban...
1,14666-138,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-08-04T11:39:19,Sim,73696,https://dadosabertos.camara.leg.br/api/v2/depu...,Angela Amin,PP,https://dadosabertos.camara.leg.br/api/v2/part...,SC,56,http://www.camara.gov.br/internet/deputado/ban...
2,14666-138,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-08-04T11:39:19,Sim,160512,https://dadosabertos.camara.leg.br/api/v2/depu...,Aureo Ribeiro,SOLIDARIEDADE,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,56,http://www.camara.gov.br/internet/deputado/ban...
3,14666-138,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-08-04T11:39:19,Sim,204374,https://dadosabertos.camara.leg.br/api/v2/depu...,Bia Kicis,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,DF,56,http://www.camara.gov.br/internet/deputado/ban...
4,14666-138,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-08-04T11:39:19,Sim,141513,https://dadosabertos.camara.leg.br/api/v2/depu...,Bilac Pinto,DEM,https://dadosabertos.camara.leg.br/api/v2/part...,MG,56,http://www.camara.gov.br/internet/deputado/ban...


In [ ]:
base_votos.isnull().sum()

idVotacao                 0
uriVotacao                0
dataHoraVoto              0
voto                      0
deputado_id               0
deputado_uri              0
deputado_nome             0
deputado_siglaPartido     0
deputado_uriPartido       0
deputado_siglaUf          0
deputado_idLegislatura    0
deputado_urlFoto          0
dtype: int64

In [ ]:
base_votos['deputado_id'].nunique()

541

In [ ]:
base_votos['idVotacao'].nunique()

824

In [ ]:
print(base_votos.voto.value_counts())

Sim          154049
Não          129597
Obstrução      2741
Abstenção       658
Artigo 17       626
Name: voto, dtype: int64


In [ ]:
propostas_count = base_votos.idVotacao.value_counts()

In [ ]:
propostas_count.head()

2272137-133    503
2270433-32     497
2272137-147    497
2272137-224    496
2272137-156    496
Name: idVotacao, dtype: int64

Preparando Base de Dados para o K-Means

Dropando informações que não vamos usar

In [ ]:
x = base_votos.drop(['uriVotacao','dataHoraVoto','uriVotacao','deputado_uri'
                   ,'deputado_uriPartido','deputado_siglaUf'
                   ,'deputado_idLegislatura','deputado_urlFoto'],axis=1)

Transformando votos em 0 (Não) e 1 (Sim)

In [ ]:
x = x.drop(x[(x.voto == 'Obstrução') | (x.voto == 'Abstenção') | (x.voto == 'Artigo 17')].index)
print(x.voto.value_counts())

x.replace({'voto':{'Sim':1,'Não':0}},inplace=True)

x = x.pivot_table('voto', ['deputado_id','deputado_nome','deputado_siglaPartido'], 'idVotacao')


Sim    154049
Não    129597
Name: voto, dtype: int64


In [ ]:
x.columns

Index(['idVotacao', 'voto', 'deputado_id', 'deputado_nome',
       'deputado_siglaPartido'],
      dtype='object')

In [ ]:
x.head()

,idVotacao,voto,deputado_id,deputado_nome,deputado_siglaPartido
0,14666-138,Sim,204545,Alê Silva,PSL
1,14666-138,Sim,73696,Angela Amin,PP
2,14666-138,Sim,160512,Aureo Ribeiro,SOLIDARIEDADE
3,14666-138,Sim,204374,Bia Kicis,PSL
4,14666-138,Sim,141513,Bilac Pinto,DEM


Para selecionar as votacoes que vamos usar na clusterização vamos usar um critério de número de votos. Conseguimos clusterizar apenas os deputados que votaram em todas as votações selecionadas, logo se selecionamos votações com baixo quorum, conseguimos clusterizar poucos deputados.
Este critério foi utilizado para este exemplo mas você pode escolher qualquer conjunto de votações que deseja analisar.

In [ ]:
propostas_selecionadas = propostas_count[propostas_count > 495]

In [ ]:
propostas_selecionadas = list(propostas_selecionadas.index)

In [ ]:
print(propostas_selecionadas)

['2272137-133', '2270433-32', '2272137-147', '2272137-224', '2272137-156', '2293449-241']


In [ ]:
base_votacao = pd.read_csv("votacoesObjetos-2021.csv",sep=';')

In [ ]:
base_votacao_selecionadas = base_votacao[base_votacao['idVotacao'].isin(propostas_selecionadas)]


In [ ]:
base_votacao_selecionadas.head()

In [ ]:
print(base_votacao_selecionadas['proposicao_ementa'].values[2])

Altera os arts. 29-A, 37, 49, 84, 163, 165, 167, 168 e 169 da Constituição Federal e os arts. 101 e 109 do Ato das Disposições Constitucionais Transitórias; acrescenta à Constituição Federal os arts. 164-A, 167-A, 167-B, 167-C, 167-D, 167-E, 167-F e 167-G; revoga dispositivos do Ato das Disposições Constitucionais Transitórias e institui regras transitórias sobre redução de benefícios tributários; desvincula parcialmente o superávit financeiro de fundos públicos; e suspende condicionalidades para realização de despesas com concessão de auxílio emergencial residual para enfrentar as consequências sociais e econômicas da pandemia de Covid-19.


Abrir o arquivo csv ou excel com as votações pode ser mais fácil para olhar a pauta de cada uma

Criando base final para a segmentação - Aqui você pode selecionar as votacoes que preferir para a clusterização, basta criar uma nova lista com outras votacoes de interesse.

In [ ]:
x1 = x[propostas_selecionadas]

In [ ]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 553 entries, (62881, 'Danilo Forte', 'PSDB') to (218086, 'Ely Santos', 'REPUBLICANOS')
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2272137-133  500 non-null    float64
 1   2270433-32   494 non-null    float64
 2   2272137-147  496 non-null    float64
 3   2272137-224  493 non-null    float64
 4   2272137-156  495 non-null    float64
 5   2293449-241  495 non-null    float64
dtypes: float64(6)
memory usage: 37.5+ KB


In [ ]:
x1 = x1.dropna()

In [ ]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 423 entries, (62881, 'Danilo Forte', 'PSDB') to (215045, 'Pedro Augusto Palareti', 'PSD')
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2272137-133  423 non-null    float64
 1   2270433-32   423 non-null    float64
 2   2272137-147  423 non-null    float64
 3   2272137-224  423 non-null    float64
 4   2272137-156  423 non-null    float64
 5   2293449-241  423 non-null    float64
dtypes: float64(6)
memory usage: 30.8+ KB


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(init="random", n_clusters=3, n_init=10,max_iter=300)

In [ ]:
labels = kmeans.fit(x1)
prediction = kmeans.predict(x1)
prediction = pd.DataFrame(prediction)

In [ ]:
x1 = x1.reset_index(level =['deputado_id','deputado_nome','deputado_siglaPartido'])
x1['Cluster'] = prediction

In [ ]:
x1.head()

idVotacao,deputado_id,deputado_nome,deputado_siglaPartido,2272137-133,2270433-32,2272137-147,2272137-224,2272137-156,2293449-241,Cluster
0,62881,Danilo Forte,PSDB,1.0,0.0,1.0,1.0,1.0,0.0,2
1,66179,Norma Ayub,DEM,0.0,0.0,1.0,1.0,1.0,1.0,2
2,66828,Fausto Pinato,PP,0.0,1.0,0.0,0.0,0.0,1.0,0
3,67138,Iracema Portella,PP,1.0,1.0,1.0,1.0,1.0,1.0,1
4,68720,Fábio Henrique,PDT,0.0,1.0,0.0,0.0,0.0,0.0,0


Proporção de partidos x classes

In [ ]:
teste = x1.groupby(['deputado_siglaPartido','Cluster']).size().reset_index(name='counts')


In [ ]:
teste.head()

,deputado_siglaPartido,Cluster,counts
0,AVANTE,0,2
1,AVANTE,1,5
2,AVANTE,2,1
3,CIDADANIA,1,4
4,CIDADANIA,2,1


In [ ]:
teste1 = teste.pivot_table('counts', ['deputado_siglaPartido'], 'Cluster')


In [ ]:
teste1.head()

Cluster,0,1,2
deputado_siglaPartido,,,
AVANTE,2.0,5.0,1.0
CIDADANIA,NaN,4.0,1.0
DEM,NaN,18.0,4.0
MDB,2.0,20.0,5.0
NOVO,NaN,NaN,8.0


Exportando para analisar resultados

In [ ]:
x1.to_csv("clusters_deputados.csv", sep=';')